# Youtube Data Cleaning

This file was used to clean the Youtube data obtained from Google Takeout. The cleaned data and results are saved as a csv file to be used in sleep_inference.ipynb.

In [725]:
# Import necessary libraries
import pandas as pd
import json
import numpy as np
from datetime import datetime, timedelta

## Clean the watch history data

In [726]:
# REPLACE WITH YOUR FILE PATH
filename = "../EZ_data/watch-history-json.json"

with open(filename, 'r', encoding='utf-8') as file:
    data = json.load(file)

# List to store extracted data
extracted_data = []

# Loop through each entry in the JSON data and extract the relevant fields
for entry in data:
    title = entry.get("title", None)
    url = entry.get("titleUrl", None)
    time_str = entry.get("time", None)

    # Try to parse the date (you can adjust the format if needed)
    date_time = None
    if time_str:
        try:
            date_time = datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
        except ValueError:
            date_time = time_str  # Keep original string if parsing fails

    # Only append if there is a valid title (to avoid extra records)
    if title:
        extracted_data.append({"title": title, "url": url, "date_time": date_time})

# Convert the extracted data to a DataFrame
df_watch = pd.DataFrame(extracted_data)
df_watch.head()


,title,url,date_time
0,Watched Square: Moulin Rouge! The Musical | Al...,https://www.youtube.com/watch?v=20ULlzE8D9M,2025-02-23 22:14:52.443000
1,Watched What happened to Asian Girl Squad??,https://www.youtube.com/watch?v=y-IAIvuorwo,2025-02-23 19:59:30.185000
2,Watched Moulin Rouge! The Musical | Altria The...,https://www.youtube.com/watch?v=2HsgXO5RU-o,2025-02-23 19:59:17.681000
3,Watched Ray-Ban Meta Big Game | Where’s My Car?,https://www.youtube.com/watch?v=WM3ugxIZvlg,2025-02-23 19:59:01.638000
4,Watched CLIF Bar | Raise Your Bar | Showing Up,https://www.youtube.com/watch?v=gJzKraNhd3s,2025-02-23 19:58:49.003000


In [727]:
# Convert date_time to datetime
df_watch['date_time'] = pd.to_datetime(df_watch['date_time'], errors='coerce', utc=True)

# Convert from UTC to US/Eastern timezone
df_watch['date_time'] = df_watch['date_time'].dt.tz_convert('US/Eastern')

# Extract the date and time separately
df_watch['date'] = df_watch['date_time'].dt.date
df_watch['time'] = df_watch['date_time'].dt.time
df_watch.head(2)

,title,url,date_time,date,time
0,Watched Square: Moulin Rouge! The Musical | Al...,https://www.youtube.com/watch?v=20ULlzE8D9M,2025-02-23 17:14:52.443000-05:00,2025-02-23,17:14:52.443000
1,Watched What happened to Asian Girl Squad??,https://www.youtube.com/watch?v=y-IAIvuorwo,2025-02-23 14:59:30.185000-05:00,2025-02-23,14:59:30.185000


In [728]:
# Extract title
pattern = r'^(Watched)\s+(.*)$'

df_watch[["type", "title"]] = df_watch["title"].str.extract(pattern)
df_watch.head(2)

,title,url,date_time,date,time,type
0,Square: Moulin Rouge! The Musical | Altria The...,https://www.youtube.com/watch?v=20ULlzE8D9M,2025-02-23 17:14:52.443000-05:00,2025-02-23,17:14:52.443000,Watched
1,What happened to Asian Girl Squad??,https://www.youtube.com/watch?v=y-IAIvuorwo,2025-02-23 14:59:30.185000-05:00,2025-02-23,14:59:30.185000,Watched


In [729]:
# Add videos per day
videos_per_day = df_watch.groupby('date').size().reset_index(name='total_videos_watched')
videos_per_day['date'] = pd.to_datetime(videos_per_day['date'], errors='coerce')
start_date = "2025-01-01"
end_date = "2025-02-17"
start_date = pd.to_datetime(start_date).date()
start_date = pd.Timestamp(start_date)
end_date = pd.to_datetime(end_date).date()
end_date = pd.Timestamp(end_date)
videos_per_day = videos_per_day[(videos_per_day["date"] >= start_date) & (videos_per_day["date"] <= end_date)]

videos_per_day.head()

,date,total_videos_watched
2526,2025-01-01,6
2527,2025-01-02,45
2528,2025-01-03,35
2529,2025-01-04,11
2530,2025-01-05,39


## Clean the search history data

In [730]:
# REPLACE WITH YOUR FILE PATH
filename = "../EZ_data/search-history-json.json"

with open(filename, 'r', encoding='utf-8') as file:
    data = json.load(file)

# List to store extracted data
extracted_data = []

# Loop through each entry in the JSON data and extract the relevant fields
for entry in data:
    title = entry.get("title", None)
    url = entry.get("titleUrl", None)
    time_str = entry.get("time", None)

    # Try to parse the date (you can adjust the format if needed)
    date_time = None
    if time_str:
        try:
            date_time = datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
        except ValueError:
            date_time = time_str  # Keep original string if parsing fails

    # Only append if there is a valid title (to avoid extra records)
    if title:
        extracted_data.append({"title": title, "url": url, "date_time": date_time})

# Convert the extracted data to a DataFrame
df_search = pd.DataFrame(extracted_data)
df_search.head()

,title,url,date_time
0,Watched Moulin Rouge! The Musical | Altria The...,https://www.youtube.com/watch?v=2HsgXO5RU-o,2025-02-23 19:59:17.681000
1,Watched Ray-Ban Meta Big Game | Where’s My Car?,https://www.youtube.com/watch?v=WM3ugxIZvlg,2025-02-23 19:59:01.638000
2,Searched for florence sun,https://www.youtube.com/results?search_query=f...,2025-02-23 14:50:35.962000
3,Watched Marvel Studios’ Captain America: Brave...,https://www.youtube.com/watch?v=u64dcreLHpA,2025-02-21 22:43:15.691000
4,Watched Scarecrow | LiMu Emu & Doug | Liberty ...,https://www.youtube.com/watch?v=DHWKG-3sIeQ,2025-02-21 22:43:08.650000


In [731]:
# Separate the search history and watch history using regex
pattern = r'^(Searched|Watched)\s+(.*)$'

df_search[["type", "title"]] = df_search["title"].str.extract(pattern)
df_search.head()

,title,url,date_time,type
0,Moulin Rouge! The Musical | Altria Theater | M...,https://www.youtube.com/watch?v=2HsgXO5RU-o,2025-02-23 19:59:17.681000,Watched
1,Ray-Ban Meta Big Game | Where’s My Car?,https://www.youtube.com/watch?v=WM3ugxIZvlg,2025-02-23 19:59:01.638000,Watched
2,for florence sun,https://www.youtube.com/results?search_query=f...,2025-02-23 14:50:35.962000,Searched
3,Marvel Studios’ Captain America: Brave New World,https://www.youtube.com/watch?v=u64dcreLHpA,2025-02-21 22:43:15.691000,Watched
4,Scarecrow | LiMu Emu & Doug | Liberty Mutual I...,https://www.youtube.com/watch?v=DHWKG-3sIeQ,2025-02-21 22:43:08.650000,Watched


In [732]:
# Convert date_time to datetime
df_search['date_time'] = pd.to_datetime(df_search['date_time'], errors='coerce', utc=True)

# Convert from UTC to US/Eastern timezone
df_search['date_time'] = df_search['date_time'].dt.tz_convert('US/Eastern')

# Extract the date and time separately
df_search['date'] = df_search['date_time'].dt.date
df_search['time'] = df_search['date_time'].dt.time
df_search.head(2)

,title,url,date_time,type,date,time
0,Moulin Rouge! The Musical | Altria Theater | M...,https://www.youtube.com/watch?v=2HsgXO5RU-o,2025-02-23 14:59:17.681000-05:00,Watched,2025-02-23,14:59:17.681000
1,Ray-Ban Meta Big Game | Where’s My Car?,https://www.youtube.com/watch?v=WM3ugxIZvlg,2025-02-23 14:59:01.638000-05:00,Watched,2025-02-23,14:59:01.638000


## Combine the search and watch history data exported from Google Takeout

In [733]:
# Combine the dataframes
df = pd.concat([df_search, df_watch], axis=0)

# Remove duplicate records
df = df.drop_duplicates()

# Sort by date and time and filter out time range
start_date = "2025-01-01"
end_date = "2025-02-17"
start_date = pd.to_datetime(start_date).date()
end_date = pd.to_datetime(end_date).date()
df = df[(df["date"] >= start_date) & (df["date"] <= end_date)]
df = df.sort_values("date_time")

# Extract the day of the week
df['day_of_week'] = df['date_time'].dt.day_name()

df.head()

,title,url,date_time,type,date,time,day_of_week
1212,the gold titanium apple watch series 10 is abs...,https://www.youtube.com/watch?v=LdNcXiYX_GY,2025-01-01 11:24:47.221000-05:00,Watched,2025-01-01,11:24:47.221000,Wednesday
232,for apple watch series 10 women,https://www.youtube.com/results?search_query=a...,2025-01-01 11:25:12.599000-05:00,Searched,2025-01-01,11:25:12.599000,Wednesday
231,for apple watch series 10 faces,https://www.youtube.com/results?search_query=a...,2025-01-01 11:25:41.201000-05:00,Searched,2025-01-01,11:25:41.201000,Wednesday
1211,Coolest Apple Watch Face 2023: My Favorite App...,https://www.youtube.com/watch?v=DOxvy-h2RW4,2025-01-01 11:25:55.529000-05:00,Watched,2025-01-01,11:25:55.529000,Wednesday
1210,HOW I SETUP MY APPLE WATCH AKA THE BEST WATCH ...,https://www.youtube.com/watch?v=A12rAfgd8Fw,2025-01-01 11:26:02.448000-05:00,Watched,2025-01-01,11:26:02.448000,Wednesday


## Create summary for analyzing Youtube summary statistics

In [734]:
# Define a threshold for a new session; events more than 30 minutes apart indicate a new session
session_gap = timedelta(minutes=30)

# Compute the difference in time between consecutive events
df['time_diff'] = df['date_time'].diff()

# Create a boolean column indicating if an event starts a new session
df['new_session'] = (df['time_diff'] > session_gap) | (df['time_diff'].isna())

# Create a session ID by cumulatively summing the new_session markers
df['session_id'] = df['new_session'].cumsum()

# Calculate session start, end, and duration by grouping on session_id
session_stats = df.groupby('session_id')['date_time'].agg(['min', 'max'])
session_stats['session_duration'] = session_stats['max'] - session_stats['min']
session_stats['date'] = session_stats['max'].dt.date

session_stats.head()

,min,max,session_duration,date
session_id,,,,
1,2025-01-01 11:24:47.221000-05:00,2025-01-01 11:26:02.448000-05:00,0 days 00:01:15.227000,2025-01-01
2,2025-01-01 13:05:57.587000-05:00,2025-01-01 13:05:57.587000-05:00,0 days 00:00:00,2025-01-01
3,2025-01-01 17:20:04.443000-05:00,2025-01-01 17:20:12.713000-05:00,0 days 00:00:08.270000,2025-01-01
4,2025-01-02 10:42:44.410000-05:00,2025-01-02 10:43:21.343000-05:00,0 days 00:00:36.933000,2025-01-02
5,2025-01-02 12:23:30.022000-05:00,2025-01-02 12:33:29.839000-05:00,0 days 00:09:59.817000,2025-01-02


In [735]:
# Extract the date from the session's end time (max)
session_stats['date'] = session_stats['max'].dt.date

# For each date, identify the session with the latest end time ('max')
last_sessions = session_stats.loc[session_stats.groupby('date')['max'].idxmax()]

# Get the session IDs corresponding to the last session of each day
last_session_ids = last_sessions.index

# Filter the original DataFrame to include only the records from these last sessions
df_last_sessions = df[df['session_id'].isin(last_session_ids)]

# Now, for each of these last sessions, pick only the final record (i.e. the one with the max timestamp)
last_record_per_session = df_last_sessions.groupby('session_id', as_index=False).last()

last_session_df = last_record_per_session[['date_time', 'session_id', 'time_diff', 'date']]
last_session_df.head()

,date_time,session_id,time_diff,date
0,2025-01-01 17:20:12.713000-05:00,3,0 days 00:00:08.270000,2025-01-01
1,2025-01-02 22:42:36.226000-05:00,8,0 days 04:23:23.311000,2025-01-02
2,2025-01-03 13:07:38.305000-05:00,9,0 days 00:00:26.006000,2025-01-03
3,2025-01-04 09:57:56.303000-05:00,11,0 days 00:00:08.783000,2025-01-04
4,2025-01-05 22:56:39.120000-05:00,14,0 days 00:07:35.977000,2025-01-05


## Create summary for overall Youtube activity

In [736]:
# Function for getting the first and last visits on Youtube 
def get_first_and_last_visits(df):
    # find first and last visited websites per date
    first_visits = df.loc[df.groupby("date")["time"].idxmin()]
    last_visits = df.loc[df.groupby("date")["time"].idxmax()]

    # select relevant columns
    first_visits = first_visits[["date", "time", "url", "title", "type"]].rename(columns={
        "time": "time_first",
        "url": "url_first",
        "title": "title_first",
        "type": "type_first"
    })
    
    last_visits = last_visits[["date", "time", "url", "title", "type"]].rename(columns={
        "time": "time_last",
        "url": "url_last",
        "title": "title_last",
        "type": "type_last"
    })

    # merge both DataFrames on visit_date to get one row per date
    youtube_summary = pd.merge(first_visits, last_visits, on="date")
    return youtube_summary

youtube_summary_df = get_first_and_last_visits(df)

youtube_summary_df.head()

,date,time_first,url_first,title_first,type_first,time_last,url_last,title_last,type_last
0,2025-01-01,11:24:47.221000,https://www.youtube.com/watch?v=LdNcXiYX_GY,the gold titanium apple watch series 10 is abs...,Watched,17:20:12.713000,https://www.youtube.com/watch?v=BuMEDjtWC78,Craftsy’s 99% OFF New Year's Crochet Deal.,Watched
1,2025-01-02,10:42:44.410000,https://www.youtube.com/results?search_query=m...,for moss stitch scarf,Searched,22:42:36.226000,https://www.youtube.com/watch?v=oTbDmSTwZN4,4.0 | Gifting | EN - US | YouTube In-Feed | Ch...,Watched
2,2025-01-03,12:47:15.706000,https://www.youtube.com/watch?v=SRCZ-59k4UA,DMV: Online Degrees & Certificates_App Fee wai...,Watched,13:07:38.305000,https://www.youtube.com/watch?v=-g4dQt57OKo,Bro was a member 💀🤣 #kaneljoseph #funnymoments...,Watched
3,2025-01-04,00:23:09.520000,https://www.youtube.com/watch?v=eymzYuoLJ04,My reaction to big bro come handle this intrud...,Watched,09:57:56.303000,https://www.youtube.com/watch?v=fsU7Wdbcrvo,[MultiSub]没有人硬cue我们昭昭墨墨就是纯甜啊！孟子义每一次撒娇李昀锐都招架不住 ...,Watched
4,2025-01-04,00:23:09.520000,https://www.youtube.com/watch?v=eymzYuoLJ04,My reaction to big bro come handle this intrud...,Watched,09:49:30.096000,https://www.youtube.com/watch?v=5hLPYkJ5YOU,Kid-Sized,Watched


In [737]:
# Add the data for the last session each night
youtube_summary_df = pd.merge(youtube_summary_df, last_session_df, on="date", how="inner")
youtube_summary_df["had_session"] = np.where(youtube_summary_df["session_id"].isna(), False, True)
youtube_summary_df = youtube_summary_df.drop_duplicates(subset=['date'])

# Rename column for time_diff
youtube_summary_df = youtube_summary_df.rename(columns={"time_diff": "session_duration"})

youtube_summary_df.head()

,date,time_first,url_first,title_first,type_first,time_last,url_last,title_last,type_last,date_time,session_id,session_duration,had_session
0,2025-01-01,11:24:47.221000,https://www.youtube.com/watch?v=LdNcXiYX_GY,the gold titanium apple watch series 10 is abs...,Watched,17:20:12.713000,https://www.youtube.com/watch?v=BuMEDjtWC78,Craftsy’s 99% OFF New Year's Crochet Deal.,Watched,2025-01-01 17:20:12.713000-05:00,3,0 days 00:00:08.270000,True
1,2025-01-02,10:42:44.410000,https://www.youtube.com/results?search_query=m...,for moss stitch scarf,Searched,22:42:36.226000,https://www.youtube.com/watch?v=oTbDmSTwZN4,4.0 | Gifting | EN - US | YouTube In-Feed | Ch...,Watched,2025-01-02 22:42:36.226000-05:00,8,0 days 04:23:23.311000,True
2,2025-01-03,12:47:15.706000,https://www.youtube.com/watch?v=SRCZ-59k4UA,DMV: Online Degrees & Certificates_App Fee wai...,Watched,13:07:38.305000,https://www.youtube.com/watch?v=-g4dQt57OKo,Bro was a member 💀🤣 #kaneljoseph #funnymoments...,Watched,2025-01-03 13:07:38.305000-05:00,9,0 days 00:00:26.006000,True
3,2025-01-04,00:23:09.520000,https://www.youtube.com/watch?v=eymzYuoLJ04,My reaction to big bro come handle this intrud...,Watched,09:57:56.303000,https://www.youtube.com/watch?v=fsU7Wdbcrvo,[MultiSub]没有人硬cue我们昭昭墨墨就是纯甜啊！孟子义每一次撒娇李昀锐都招架不住 ...,Watched,2025-01-04 09:57:56.303000-05:00,11,0 days 00:00:08.783000,True
5,2025-01-05,19:34:17.228000,https://www.youtube.com/watch?v=SRCZ-59k4UA,DMV: Online Degrees & Certificates_App Fee wai...,Watched,22:56:39.120000,https://www.youtube.com/watch?v=TDBNltzAg_w,=ENG SUB=天盛長歌 The Rise of Phoenixes 02 陳坤 倪妮 C...,Watched,2025-01-05 22:56:39.120000-05:00,14,0 days 00:07:35.977000,True


## Combine Youtube summary with sleep data

In [738]:
# Import sleep data

# CHANGE THIS TO YOUR FILE PATH
sleep_filename = "../elaine_sleep_data.csv"

sleep_df = pd.read_csv(sleep_filename)
sleep_df["Date"] = pd.to_datetime(sleep_df["Date"])
sleep_df = sleep_df.rename(columns={"Date": "date"})
sleep_df = sleep_df.drop(columns=["Unnamed: 0"])
sleep_df.tail()

,date,total_sleep_hours,sleep_start,sleep_end
1228,2025-02-13,7.650000,2025-02-13 00:51:48-05:00,2025-02-13 08:40:48-05:00
1229,2025-02-14,8.316667,2025-02-14 01:20:56-05:00,2025-02-14 09:59:26-05:00
1230,2025-02-15,7.041667,2025-02-15 02:19:59-05:00,2025-02-15 09:28:59-05:00
1231,2025-02-16,7.791667,2025-02-16 01:50:12-05:00,2025-02-16 09:56:42-05:00
1232,2025-02-17,8.408333,2025-02-17 00:55:21-05:00,2025-02-17 09:59:51-05:00


In [739]:
youtube_summary_df["date"] = pd.to_datetime(youtube_summary_df["date"])
df = pd.merge(sleep_df, youtube_summary_df, on="date", how="right")
df.head()

,date,total_sleep_hours,sleep_start,sleep_end,time_first,url_first,title_first,type_first,time_last,url_last,title_last,type_last,date_time,session_id,session_duration,had_session
0,2025-01-01,7.566667,2025-01-01 01:58:50-05:00,2025-01-01 10:00:20-05:00,11:24:47.221000,https://www.youtube.com/watch?v=LdNcXiYX_GY,the gold titanium apple watch series 10 is abs...,Watched,17:20:12.713000,https://www.youtube.com/watch?v=BuMEDjtWC78,Craftsy’s 99% OFF New Year's Crochet Deal.,Watched,2025-01-01 17:20:12.713000-05:00,3,0 days 00:00:08.270000,True
1,2025-01-02,6.608333,2025-01-02 02:10:26-05:00,2025-01-02 08:59:56-05:00,10:42:44.410000,https://www.youtube.com/results?search_query=m...,for moss stitch scarf,Searched,22:42:36.226000,https://www.youtube.com/watch?v=oTbDmSTwZN4,4.0 | Gifting | EN - US | YouTube In-Feed | Ch...,Watched,2025-01-02 22:42:36.226000-05:00,8,0 days 04:23:23.311000,True
2,2025-01-03,NaN,NaN,NaN,12:47:15.706000,https://www.youtube.com/watch?v=SRCZ-59k4UA,DMV: Online Degrees & Certificates_App Fee wai...,Watched,13:07:38.305000,https://www.youtube.com/watch?v=-g4dQt57OKo,Bro was a member 💀🤣 #kaneljoseph #funnymoments...,Watched,2025-01-03 13:07:38.305000-05:00,9,0 days 00:00:26.006000,True
3,2025-01-04,7.875000,2025-01-04 01:22:05-05:00,2025-01-04 09:34:35-05:00,00:23:09.520000,https://www.youtube.com/watch?v=eymzYuoLJ04,My reaction to big bro come handle this intrud...,Watched,09:57:56.303000,https://www.youtube.com/watch?v=fsU7Wdbcrvo,[MultiSub]没有人硬cue我们昭昭墨墨就是纯甜啊！孟子义每一次撒娇李昀锐都招架不住 ...,Watched,2025-01-04 09:57:56.303000-05:00,11,0 days 00:00:08.783000,True
4,2025-01-05,8.775000,2025-01-05 00:32:58-05:00,2025-01-05 09:33:28-05:00,19:34:17.228000,https://www.youtube.com/watch?v=SRCZ-59k4UA,DMV: Online Degrees & Certificates_App Fee wai...,Watched,22:56:39.120000,https://www.youtube.com/watch?v=TDBNltzAg_w,=ENG SUB=天盛長歌 The Rise of Phoenixes 02 陳坤 倪妮 C...,Watched,2025-01-05 22:56:39.120000-05:00,14,0 days 00:07:35.977000,True


In [740]:
# Add videos per day
df = pd.merge(df, videos_per_day,  on="date", how="right")
df = df[df['total_videos_watched'] <= 180]
df.head()

,date,total_sleep_hours,sleep_start,sleep_end,time_first,url_first,title_first,type_first,time_last,url_last,title_last,type_last,date_time,session_id,session_duration,had_session,total_videos_watched
0,2025-01-01,7.566667,2025-01-01 01:58:50-05:00,2025-01-01 10:00:20-05:00,11:24:47.221000,https://www.youtube.com/watch?v=LdNcXiYX_GY,the gold titanium apple watch series 10 is abs...,Watched,17:20:12.713000,https://www.youtube.com/watch?v=BuMEDjtWC78,Craftsy’s 99% OFF New Year's Crochet Deal.,Watched,2025-01-01 17:20:12.713000-05:00,3,0 days 00:00:08.270000,True,6
1,2025-01-02,6.608333,2025-01-02 02:10:26-05:00,2025-01-02 08:59:56-05:00,10:42:44.410000,https://www.youtube.com/results?search_query=m...,for moss stitch scarf,Searched,22:42:36.226000,https://www.youtube.com/watch?v=oTbDmSTwZN4,4.0 | Gifting | EN - US | YouTube In-Feed | Ch...,Watched,2025-01-02 22:42:36.226000-05:00,8,0 days 04:23:23.311000,True,45
2,2025-01-03,NaN,NaN,NaN,12:47:15.706000,https://www.youtube.com/watch?v=SRCZ-59k4UA,DMV: Online Degrees & Certificates_App Fee wai...,Watched,13:07:38.305000,https://www.youtube.com/watch?v=-g4dQt57OKo,Bro was a member 💀🤣 #kaneljoseph #funnymoments...,Watched,2025-01-03 13:07:38.305000-05:00,9,0 days 00:00:26.006000,True,35
3,2025-01-04,7.875000,2025-01-04 01:22:05-05:00,2025-01-04 09:34:35-05:00,00:23:09.520000,https://www.youtube.com/watch?v=eymzYuoLJ04,My reaction to big bro come handle this intrud...,Watched,09:57:56.303000,https://www.youtube.com/watch?v=fsU7Wdbcrvo,[MultiSub]没有人硬cue我们昭昭墨墨就是纯甜啊！孟子义每一次撒娇李昀锐都招架不住 ...,Watched,2025-01-04 09:57:56.303000-05:00,11,0 days 00:00:08.783000,True,11
4,2025-01-05,8.775000,2025-01-05 00:32:58-05:00,2025-01-05 09:33:28-05:00,19:34:17.228000,https://www.youtube.com/watch?v=SRCZ-59k4UA,DMV: Online Degrees & Certificates_App Fee wai...,Watched,22:56:39.120000,https://www.youtube.com/watch?v=TDBNltzAg_w,=ENG SUB=天盛長歌 The Rise of Phoenixes 02 陳坤 倪妮 C...,Watched,2025-01-05 22:56:39.120000-05:00,14,0 days 00:07:35.977000,True,39


In [741]:
# RENAME FILE AS APPROPRIATE
df.to_csv("../youtube_sleep_EZ.csv")